# 1. 강의영상 

# 2. ref 

ref: <https://huggingface.co/docs/transformers/tasks/image_classification>

# 3. imports 

In [1]:
import datasets
import transformers
import torchvision.transforms
import evaluate
import numpy as np

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 4. 코드정리 

In [11]:
common_voice = datasets.load_dataset("mozilla-foundation/common_voice_11_0", "en")

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 0 examples [00:00, ? examples/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 37512it [00:00, 375102.47it/s]
Reading metadata...: 75023it [00:00, 369365.98it/s]
Reading metadata...: 111966it [00:00, 363426.94it/s]
Reading metadata...: 148319it [00:00, 360443.91it/s]
Reading metadata...: 184369it [00:00, 353460.10it/s]
Reading metadata...: 219732it [00:00, 352603.80it/s]
Reading metadata...: 255003it [00:00, 3

In [10]:
## Step1 
minds = datasets.load_dataset("PolyAI/minds14", name="en-US", split="train")
minds = minds.train_test_split(test_size=0.2)
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])
minds = minds.cast_column("audio", datasets.Audio(sampling_rate=16_000))
feature_extractor = transformers.AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")
## Step2 
labels = minds["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
num_labels = len(id2label)
model = transformers.AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)
## Step3 
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)
training_args = transformers.TrainingArguments(
    output_dir="my_awesome_mind_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    #warmup_ratio=0.1,
    warmup_steps=500,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)
trainer.train()
## Step4 

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/configuration_utils.py:302: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████

Epoch,Training Loss,Validation Loss,Accuracy
0,2.639700,2.639443,0.088496
2,2.637600,2.640823,0.088496
4,2.635000,2.643230,0.070796
6,2.632800,2.640821,0.079646
8,2.630400,2.644272,0.079646
9,2.626900,2.643184,0.079646


/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: i

TrainOutput(global_step=140, training_loss=2.6340601103646413, metrics={'train_runtime': 54.9468, 'train_samples_per_second': 81.897, 'train_steps_per_second': 2.548, 'total_flos': 3.9529655577216e+16, 'train_loss': 2.6340601103646413, 'epoch': 9.655172413793103})

In [9]:
# Step4 
dataset = datasets.load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]
classifier = transformers.pipeline("audio-classification", model="my_awesome_mind_model/checkpoint-140")
classifier(audio_file)

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.07800574600696564, 'label': 'cash_deposit'},
 {'score': 0.07708185911178589, 'label': 'freeze'},
 {'score': 0.0737047791481018, 'label': 'business_loan'},
 {'score': 0.07282004505395889, 'label': 'direct_debit'},
 {'score': 0.0727161318063736, 'label': 'atm_limit'}]

# 5. 살펴보기 